<a href="https://colab.research.google.com/github/jhaarun/jhaarun.github.io/blob/master/Training_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Dec 17 08:49:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'193109017 (1).zip'
 193109017.zip
'3rd sem fees payment.JPG'
'A2_example_dataset - students_demo.csv'
'aadhar card_updated.JPG'
'Adobe Scan 04-Oct-2020 (1).pdf'
'afDRDO_Resume[1].pdf'
 afDRDO_Resume.docx
 afDRDO_Resume.docx.gdoc
 Air_pods.jfif
 Aptitude.zip
'Baby Incubator'
 Bonafide.pdf
 C100A34AdmitCard.pdf
'Colab Notebooks'
'en office professional plus 2013 x86 dvd 1123673[Bellatrix].iso'
 Gate_2018scoreCard.pdf
 Gateme.jpg
'HSC-1[1].pdf'
 IMG_20160206_110153.jpg
 IMG_20201209_123846.jpg
 LH.gdoc
 MacBook-Air.png
'Made Easy Aptitude and Reasoning.pdf'
'ME781: End Sem'
'ME 781 Project'
'My account info.zip'
'New Doc 2018-11-23 23.56.05.pdf'
'Object Detection'
'Patent Search_EEJM'
 pen.jpeg
'photo (1).jpg'
'Photo and sign'
 photo.jpg
 pic.jpg
 Portfolio.gsite
 Project_Report_Final_edited_ajay1.pdf
 Resume_02.docx
 Resume_02.pdf
'Scanned cetificates'
 SIGN_LTG.jpg
 sign.pdf
'SOM 630+MSP+Social Enterprie Analysis+Manuvikasa.pptx'
 SSC.pdf
 stairs.jfif
'WhatsA

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14639 (delta 4), reused 8 (delta 3), pack-reused 14621
Receiving objects: 100% (14639/14639), 13.25 MiB | 25.17 MiB/s, done.
Resolving deltas: 100% (9959/9959), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "pwm" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-17 08:52:44--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   400KB/s    in 7m 48s  

2020-12-17 09:00:32 (339 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/download (1).jpg  
 extracting: data/obj/download (1).txt  
  inflating: data/obj/download (10).jpg  
 extracting: data/obj/download (10).txt  
  inflating: data/obj/download (2).jpg  
  inflating: data/obj/download (2).txt  
  inflating: data/obj/download (3).jpg  
 extracting: data/obj/download (3).txt  
  inflating: data/obj/download (4).jpg  
 extracting: data/obj/download (4).txt  
  inflating: data/obj/download (5).jpg  
  inflating: data/obj/download (5).txt  
  inflating: data/obj/download (6).jpg  
  inflating: data/obj/download (6).txt  
  inflating: data/obj/download (7).jpg  
  inflating: data/obj/download (7).txt  
  inflating: data/obj/download (8).jpg  
  inflating: data/obj/download (8).txt  
  inflating: data/obj/download (9).jpg  
  inflating: data/obj/download (9).txt  
  inflating: data/obj/download.jpg   
 extracting: data/obj/download.txt   
  inflating: data/obj/healt

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.642241 0.384793 0.275862 0.382488

['15', '0.642241', '0.384793', '0.275862', '0.382488']
0 data/obj/download (3).txt
0 0.642241 0.384793 0.275862 0.382488
15 0.488333 0.482143 0.430000 0.750000

['15', '0.488333', '0.482143', '0.430000', '0.750000']
1 data/obj/download (8).txt
0 0.488333 0.482143 0.430000 0.750000
15 0.569355 0.220859 0.158065 0.355828

['15', '0.569355', '0.220859', '0.158065', '0.355828']
2 data/obj/images (6).txt
0 0.569355 0.220859 0.158065 0.355828
15 0.809677 0.334356 0.096774 0.214724

['15', '0.809677', '0.334356', '0.096774', '0.214724']
2 data/obj/images (6).txt
0 0.809677 0.334356 0.096774 0.214724
15 0.603774 0.283019 0.213836 0.477987

['15', '0.603774', '0.283019', '0.213836', '0.477987']
3 data/obj/download (1).txt
0 0.603774 0.283019 0.213836 0.477987
15 0.660959 0.313953 0.253425 0.488372

['15', '0.660959', '0.313953', '0.253425', '0.488372']
4 data/obj/download.txt
0 0.660959 0.313953 0.253425 0.488372
15 0.275556 0.497778 0.435556 0.488889

['

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/images (14).jpg', 'data/obj/download (7).jpg', 'data/obj/images (17).jpg', 'data/obj/images (3).jpg', 'data/obj/images (7).jpg', 'data/obj/images (26).jpg', 'data/obj/images (22).jpg', 'data/obj/images (2).jpg', 'data/obj/images (13).jpg', 'data/obj/download (4).jpg', 'data/obj/images.jpg', 'data/obj/images (4).jpg', 'data/obj/download (1).jpg', 'data/obj/images (27).jpg', 'data/obj/images (19).jpg', 'data/obj/download (2).jpg', 'data/obj/images (15).jpg', 'data/obj/images (12).jpg', 'data/obj/images (18).jpg', 'data/obj/images (28).jpg', 'data/obj/download.jpg', 'data/obj/images (1).jpg', 'data/obj/health-effects-of-air-pollution.jpg', 'data/obj/download (5).jpg', 'data/obj/images (5).jpg', 'data/obj/images (20).jpg', 'data/obj/images (11).jpg', 'data/obj/download (6).jpg', 'data/obj/images (9).jpg', 'data/obj/images (31).jpg', 'data/obj/images (21).jpg', 'data/obj/images (6).jpg', 'data/obj/download (9).jpg', 'data/obj/images (25).jpg', 'data/obj/images (16).jpg', 'data/ob

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 33.840218, iou_loss = 0.000000, total_loss = 33.840218 
 total_bbox = 10544, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.218099), count: 3, class_loss = 38.148537, iou_loss = 4.649506, total_loss = 42.798042 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 38.250816, iou_loss = 0.000000, total_loss = 38.250816 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.298924), count: 1, class_loss = 34.389778, iou_loss = 0.886936, total_loss = 35.276714 
 total_bbox = 10548, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 36.870747, iou_loss = 0.000000, total_loss = 36.87074